In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import re
import time

In [47]:
def url_state(offset, state):
    return 'https://data.cms.gov/tools/medicare-inpatient-hospital-look-up-tool?size=100&offset=' + str(offset) + '&state=' + str(state)

states = [
    'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
    'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
    'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
    'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
    'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'
    ]

service = Service(ChromeDriverManager().install())
service.SuppressInitialDiagnosticInformation = True

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\email\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache


In [48]:
driver = webdriver.Chrome(service = service)

In [ ]:
# driver.get(url_state(0, "AL"))
# time.sleep(2)
# table = driver.find_element(By.XPATH, "/html/body/div/div/div/div/div/div/div[5]/div[1]/div/div/div[2]").text.split('\n')
# # print(table.text)
# for i in [3 * x for x in range(len(table) // 3)]:
#     print(table[i])
#     print(table[i+1])
#     print(table[i+2])

In [52]:
total_results = []

for state in states:
    # load state page
    driver.get(url_state(0, state))
    time.sleep(1.5)
    # obtain number of results for the state
    no_of_results = int(driver.find_element(by = By.XPATH, value = "/html/body/div/div/div/div/div/div/div[5]/div[1]/div/div/div[1]/div[1]/span[2]").text)
    
    state_results = []

    # get every page in this state's results
    for offset in [100 * x for x in range(no_of_results // 100 + 1)]:
        page = []
        driver.get(url_state(offset, state))
        time.sleep(1.5) # wait for page to load
        results = driver.find_element(By.XPATH, "/html/body/div/div/div/div/div/div/div[5]/div[1]/div/div/div[2]").text.split('\n')
        for i in [3 * x for x in range(len(results) // 3)]:
            row = []
            row.append(results[i])
            row.append(results[i+1])
            row.append(results[i+2])
            page.append(row)
        state_results = state_results + page
        print(state + " done!", end='\r')
    
    total_results = total_results + state_results
        


In [54]:
total_results
pd.DataFrame(total_results).to_csv("Facilities_1.csv")

Clean the data in R and import it:

In [57]:
facilities = pd.read_csv("facilities_1_clean.csv", dtype='string')
facilities

,Name,City,State,CCN
0,Alaska Native Medical Center,Anchorage,AK,020026
1,Alaska Regional Hospital,Anchorage,AK,020017
2,Bartlett Regional Hospital,Juneau,AK,020008
3,Central Peninsula General Hospital,Soldotna,AK,020024
4,Fairbanks Memorial Hospital,Fairbanks,AK,020012
...,...,...,...,...
3117,Sagewest Health Care,Riverton,WY,530008
3118,Sheridan Memorial Hospital,Sheridan,WY,530006
3119,St Johns Medical Center,Jackson,WY,530015
3120,Summit Medical Center,Casper,WY,530034
